In [3]:
from pyModbusTCP.client import ModbusClient

# 建立 Modbus 客戶端
client = ModbusClient(host="127.0.0.1", port=502)


if not client.is_open:
    if not client.open():
        print("無法連接到 Modbus Server")
    else:
        print("已連接到 Modbus Server")


start = 0  # 起始位址
count = 10  # 要讀取的數量

result = client.read_holding_registers(start, count) # 讀取Register
# result = client.read_coils(start, count) # 讀取Coil

if result:
    print("讀取的值：", result)
else:
    print("Modbus 讀取失敗")

client.close()

已連接到 Modbus Server
讀取的值： [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [4]:
from pyModbusTCP.client import ModbusClient

# 建立 Modbus 客戶端
client = ModbusClient(host="127.0.0.1", port=502)

if not client.is_open:
    if not client.open():
        print("無法連接到 Modbus Server")
    else:
        print("已連接到 Modbus Server")

address = 5  # 要寫入的位址
value_to_write = 10  # 要寫入的值


result = client.write_single_register(address, value_to_write) # 寫入Register
# result = client.write_single_coil(address, value_to_write) # 寫入Coil

if result:
    print("寫入成功")
else:
    print("寫入失敗")

client.close()

已連接到 Modbus Server
寫入成功


In [10]:
import scapy.all as scapy
import scapy.contrib.modbus as mb

for pkt in scapy.PcapReader("write.pcapng"):
    if mb.ModbusPDU06WriteSingleRegisterResponse in pkt:
        pkt.show()

###[ Loopback ]### 
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 52
     id        = 27499
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0x0
     src       = 192.168.50.197
     dst       = 192.168.50.197
     \options   \
###[ TCP ]### 
        sport     = 502
        dport     = 56944
        seq       = 1680003329
        ack       = 3258498429
        dataofs   = 5
        reserved  = 0
        flags     = PA
        window    = 8442
        chksum    = 0x1192
        urgptr    = 0
        options   = []
###[ ModbusADU ]### 
           transId   = 0x2
           protoId   = 0x0
           len       = 6
           unitId    = 0x1
###[ Write Single Register Response ]### 
              funcCode  = 0x6
              registerAddr= 0x4
              registerValue= 0x6



In [17]:
from scapy.all import *

# Modbus ADU
class ModbusTCP(Packet):
    name = "Modbus/TCP"
    fields_desc = [
        ShortField("Transaction_Identifier", 3),
        ShortField("Protocol_Identifier", 0),
        ShortField("Length", 6),
        XByteField("Unit_Identifier", 1),
    ]

# Modbus PDU
class Modbus(Packet):
    name = "Modbus"
    fields_desc = [
        XByteField("Function_Code", 6),  
        ShortField("Reference_Number", 53),
        # ShortField("Word_Count", 63),
        ShortField("Data", 1234),
    ]


s = socket.socket()
s.connect(("192.168.50.197", 502))
ss = StreamSocket(s, Raw)

try:
    response = ss.sr1(Raw(ModbusTCP()/Modbus()))
    if response:
        response_hexdump = hexdump(response, dump=True)
        print(response_hexdump)

except KeyboardInterrupt:
    pass

Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
0000  00 03 00 00 00 06 01 06 00 35 04 D2              .........5..


In [18]:
from scapy.all import *

# Modbus ADU
class ModbusTCP(Packet):
    name = "Modbus/TCP"
    fields_desc = [
        ShortField("Transaction_Identifier", 3),
        ShortField("Protocol_Identifier", 0),
        ShortField("Length", 6),
        XByteField("Unit_Identifier", 1),
    ]

# Modbus PDU
class Modbus(Packet):
    name = "Modbus"
    fields_desc = [
        XByteField("Function_Code", 3),  
        ShortField("Reference_Number", 0),
        ShortField("Word_Count", 64),
        # ShortField("Data", 1234),
    ]


s = socket.socket()
s.connect(("192.168.50.197", 502))
ss = StreamSocket(s, Raw)

try:
    response = ss.sr1(Raw(ModbusTCP()/Modbus()))
    if response:
        response_hexdump = hexdump(response, dump=True)
        print(response_hexdump)

except KeyboardInterrupt:
    pass

Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
0000  00 03 00 00 00 83 01 03 80 00 00 00 00 00 00 00  ................
0010  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
0020  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
0030  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
0040  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
0050  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
0060  00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
0070  00 00 00 04 D2 00 00 00 00 00 00 00 00 00 00 00  ................
0080  00 00 00 00 00 00 00 00 00                       .........
